**Table of contents**<a id='toc0_'></a>    
- 1. [Problem 1: Optimal taxation with government consumption](#toc1_)    
- 2. [Problem 2: Labor adjustment costs](#toc2_)    
- 3. [Problem 3: Global optimizer with refined multi-start](#toc3_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [1]:
# write your code here

In [2]:
import numpy as np
import numba as nb
from types import SimpleNamespace
from scipy import optimize
# autoreload modules when code is run. Otherwise, python will not see recent changes. 
%load_ext autoreload
%autoreload 2

from scipy.optimize import root_scalar

# Import our own cod
import exam

import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


## 1. <a id='toc1_'></a>[Problem 1: Optimal taxation with government consumption](#toc0_)


Consider a worker choosing hours of labor, $L\in[0,24]$, to maximize utility: 

$$
\begin{align*}
V(w,\tau,G)&=\max_{L\in[0,24]}\ln\left(C^{\alpha}G^{1-\alpha}\right)-\nu\frac{L^{2}}{2}\\&\text{s.t.}\\&C=\kappa+(1-\tau)wL
\end{align*}
$$

where 

* $C$ is *private* consumption with weight $\alpha\in(0,1)$.
* $\kappa > 0$ is the *free private* consumption component.
* $C = (1-\tau)wL$ is the *costly private* consumption component.
* $w > 0 $ is the real wage.
* $\tau \in (0,1)$ is the labor-income tax rate.
* $G > 0 $ is *government* consumption with weight $1-\alpha$.
* $\nu > 0$ is the disutility of labor scaling factor


The baseline parameters are:

$$
\begin{align*}
\alpha &= 0.5\\
\kappa &= 1.0\\
\nu &= \frac{1}{2\cdot16^2} \\
w &= 1.0 \\ 
\tau &= 0.30 \\
\end{align*}
$$

**Question 1:** Verify that the optimal labor supply choice is $L^{\star}(\tilde{w}) =\frac{-\kappa+\sqrt{\kappa^{2}+4\frac{\alpha}{\nu}\tilde{w}^2}}{2\tilde{w}}$, where $\tilde{w} = (1-\tau)w$, for $G\in\left\{1.0 , 2.0\right\}$.

**Answer for question 1**
$$
\begin{align*}
V(w,\tau,G)=\underset{L\epsilon[0,24]}{max}ln(C^{\alpha}G^{1-\alpha})-\upsilon\frac{L^{2}}{2}\rightarrow \\
V(\overset{\sim}{\omega})=ln[(\kappa+\overset{\sim}{\omega}L)^{\alpha}G^{1-\alpha}]-\upsilon\frac{L^{2}}{2}
\end{align*}
$$
FOC: 
$$
\begin{align*}
\frac{\partial V(\overset{\sim}{\omega})}{\partial L}=0:\rightarrow \\
\frac{1}{(\kappa+\overset{\sim}{\omega}L)^{\alpha}G^{1-\alpha}}\cdot\alpha(\kappa+\overset{\sim}{\omega}L)^{\alpha-1}\cdot G^{1-\alpha}=\upsilon L\rightarrow \\
\frac{\alpha\overset{\sim}{\omega}(\kappa+\overset{\sim}{\omega}L)^{\alpha-1}}{(\kappa+\overset{\sim}{\omega}L)^{\alpha}}=\upsilon L\rightarrow \\
\frac{\alpha\overset{\sim}{\omega}}{\kappa+\overset{\sim}{\omega}L}=\upsilon L\rightarrow \\
\alpha\overset{\sim}{\omega}=\upsilon L(\kappa+\overset{\sim}{\omega}L)\rightarrow \\
\overset{\sim}{\omega}L^{2}+\kappa L-\frac{\alpha\overset{\sim}{\omega}}{\upsilon}=0
\end{align*}
$$

2nd degree polynomial:
$$
\begin{align*}
d=\kappa^{2}+4\cdot\overset{\sim}{\omega}\cdot\frac{\alpha\overset{\sim}{\omega}}{\upsilon}=\kappa^{2}+4\overset{\sim}{\omega}^{2}\frac{\alpha}{\upsilon} \\
L=\frac{-\kappa\pm\sqrt{\kappa^{2}+4\cdot\frac{\alpha}{\upsilon}\cdot\overset{\sim}{\omega}^{2}}}{2\cdot\overset{\sim}{\omega}}
\end{align*}
$$
We cannot subtract as labor supply would then be negative. Hence: 
$$
\begin{align*}
L(\overset{\sim}{\omega})=\frac{-\kappa+\sqrt{\kappa^{2}+4\cdot\frac{\alpha}{\upsilon}\cdot\overset{\sim}{\omega}^{2}}}{2\cdot\overset{\sim}{\omega}}
\end{align*}
$$


**Question 2:** Illustrate how $L^{\star}(\tilde{w})$ depends on $w$.

In [3]:
# write your code here
alpha = 0.5
kappa = 1.0
upsilon = 1/(2*16**2)
omega_vec = np.linspace(0.8,1.2,5)

In [4]:
labor = {}

for omega in omega_vec:
    L = (-kappa + np.sqrt(kappa**2 + 4*alpha/upsilon*omega**2))/(2*omega)
    labor[omega] = L 

In [5]:
#Plot labor as a function of wages. Show that as wages increases, labor supply increases. 
#Figure should resemble layout from inagural project
labor

{0.8: 15.387202378186457,
 0.9: 15.454086600823477,
 1.0: 15.507810593582121,
 1.1: 15.551909854805455,
 1.2: 15.588757761042558}


We now consider a government, who chooses $\tau$ and spend all of the taxes on government consumption so:

$$
G = \tau w L^{\star}((1-\tau)w)
$$

**Question 3:** Plot the implied $L$, $G$ and worker utility for a grid of $\tau$-values.


In [6]:
# Import taxation class model 
from exam import TaxationClass
model = TaxationClass()
par = model.par
sol = model.sol

In [7]:
# Find model solution for different taus
tau_list = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
labor = {}
utility = {}
government = {}

for tau in tau_list:
    par.tau = tau
    opt = model.solve()

    labor[tau] = opt.labor 
    utility[tau] = model.calc_utility(labor=opt.labor)
    government[tau] = model.government(labor=opt.labor)

In [8]:
# Make three different plots of labor, utiloty and government revenue as a function of tau
# write some text about the intuition of the plots
labor

{0.1: 22.358000054456244,
 0.2: 22.32546276372485,
 0.3: 22.28401455703221,
 0.4: 22.229374835759486,
 0.5: 22.154070212054496,
 0.6: 22.043687153094726,
 0.7: 21.866342062627506,
 0.8: 21.535142906446755,
 0.9: 20.703066068153362}

In [9]:
utility

{0.1: 1.4392976403106723,
 0.2: 1.729932003026946,
 0.3: 1.8696751250880426,
 0.4: 1.9414419538729109,
 0.5: 1.9687902542666937,
 0.6: 1.958646997347028,
 0.7: 1.9086353825161226,
 0.8: 1.8048943970766131,
 0.9: 1.6047781257264853}

**Question 4:** Find the socially optimal tax rate $\tau^{\star}\in(0,1)$ maximizing worker utility. Illustrate your result.

In [10]:
# write introduction to what we do: define objective function that maximizes utility under different tau values
# we do this in the py file

In [11]:
model.optimal_tax(do_print=True) 


Optimization terminated successfully.
tau_hat: 0.5226
Termination value: -1.9697


A more general preference formulation for the worker is:

$$
\begin{align*}
\mathcal{V}(w,\tau,G)&=\max_{L\in[0,24]}\frac{\left[ \left( \alpha C^{\frac{\sigma-1}{\sigma}}+(1-\alpha) G^{\frac{\sigma-1}{\sigma}} \right)^{\frac{\sigma}{1-\sigma} }\right]^{1-\rho}-1}{1-\rho}- \nu\frac{L^{1+\varepsilon}}{1+\varepsilon},\,\,\,\varepsilon,\rho,\sigma>0,\,\,\,\rho,\sigma\neq1\\&\text{s.t.}\\&C=\kappa+(1-\tau)wL
\end{align*}    
$$

Optimal labor supply is now $L^{\star}(\tilde{w},G)$.

Questions 5 and 6 must be answered with the general formulation, and for 2 different set of parameters:

- Set 1:  $\sigma = 1.001$, $\rho = 1.001$ and $\varepsilon = 1.0$.
- Set 2:  $\sigma = 1.5$, $\rho = 1.5$ and $\varepsilon = 1.0 $.

**Question 5:** Find the $G$ that solves $G = \tau w L^{\star}((1-\tau)w,G)$ using the $\tau$ found in question 4.

*Hint: First write code that solves the worker problem for given values of $G$ and $\tau$. Then find the correct G based on this.*

In [12]:
from exam import TaxationClass
model_new1 = TaxationClass()
par = model_new1.par
sol = model_new1.sol
model_new1.par.sigma = 1.001
model_new1.par.rho = 1.001
model_new1.par.tau = 0.5226

In [13]:
opt_new1 = model_new1.solve()
labor_new1 = opt_new1.labor 
utility_new1 = model_new1.calc_utility(opt_new1.labor)
government_new1 = model_new1.government(opt_new1.labor)

In [14]:
print(labor_new1)
print(utility_new1)
print(government_new1)

22.105380550909764
1.966731903764507
11.552271875905442


In [15]:
from exam import TaxationClass
model_new2 = TaxationClass()
par = model_new2.par
sol = model_new2.sol
model_new2.par.sigma = 1.5
model_new2.par.rho = 1.5
model_new2.par.tau = 0.5226

In [16]:
opt_new2 = model_new2.solve()
labor_new2 = opt_new2.labor 
utility_new2 = model_new2.calc_utility(opt_new2.labor)
government_new2 = model_new2.government(opt_new2.labor)

In [17]:
print(labor_new2)
print(utility_new2)
print(government_new2)

13.34570227609117
1.0792061591069517
6.974464009485245


**Question 6:** Find the socially optimal tax rate, $\tau^{\star}$, maximizing worker utility, while keeping $G = \tau w L^{\star}((1-\tau)w,G)$.

In [ ]:
#write intutionen to the two optimal taxes. 
# 1) why is optimal tax the same in model and model_new1?
# 2) why is utility lower in model_new2? because they work less. something about substitution. 

In [18]:
model_new1.optimal_tax(do_print=True) 

Optimization terminated successfully.
tau_hat: 0.5226
Termination value: -1.9667


In [19]:
model_new2.optimal_tax(do_print=True) 

Optimization terminated successfully.
tau_hat: 0.5375
Termination value: -1.0793


## 2. <a id='toc2_'></a>[Problem 2: Labor adjustment costs](#toc0_)

You own a hair salon. You employ hairdressers, $\ell_t$, to produce haircuts, $y_t = \ell_t$.

The wage for each haridresser is $w$.

The demand for haircuts implies that the price of haircuts you can charge is $p_t = \kappa_t y_t^{-\eta}$, where $\kappa_t$ is a demand-shock and $\eta \in (0,1)$ measures the elasticity of demand.

Profits are:

$$
\Pi_t = p_t y_t - w \ell_t = \kappa_t \ell_t^{1-\eta} - w \ell_t
$$

Baseline parameters are:
- $\eta = 0.5$
- $w = 1.0$

**Question 1:** Verify numerically that $\ell_{t}=\left(\frac{(1-\eta)\kappa_{t}}{w}\right)^{\frac{1}{\eta}}$ maximises profits, for $\kappa\in\left\{1.0 , 2.0\right\}$.

In [20]:
# write your code here

We now consider a *dynamic* version of the model.

* The demand-shock is a so-called AR(1) in logs, 

$$
\log \kappa_{t} = \rho \log \kappa_{t-1} + \epsilon_{t},\,\,\, \epsilon_{t+1} \sim \mathcal{N}(-0.5\sigma_{\epsilon}^2,\sigma_{\epsilon})
$$

* Any hiring or firing implies a fixed adjustment cost, $\iota > 0 $.
* Future profits are discounted with a monthly factor of $R \in (0,1)$.

The initial demand shock is $\kappa_{-1} = 1$ and the planning horizon is 10 years, i.e. 120 months so $t \in \{0,1,2,\dots,119\}$. Initially you don't have any employees, $\ell_{-1}=0$


The *ex post* value of the salon is *conditional* on the shock series is:

$$
h(\epsilon_0,\epsilon_1,\dots,\epsilon_{119}) = \left[\sum_{t=0}^{119}R^{-t}\left[\kappa_{t}\ell_{t}^{1-\eta}-w\ell_{t}-\boldsymbol{1}_{\ell_{t}\neq\ell_{t-1}}\iota\right]\right]
$$

The *ex ante* expected value of the salon can be approximated by

$$
H = \mathbb{E}[h(\epsilon_0,\epsilon_1,\dots,\epsilon_{119})] \approx \frac{1}{K}\sum_{k=0}^{K} h(\epsilon_0^k,\epsilon_1^k,\dots,\epsilon_{119}^k)
$$

where each $k\in\{0,1,\dots,K-1\}$ is a random shock series. Maximizing profitability means maximizing $H$.


Baseline parameters are: 

- $\rho = 0.90$
- $\iota = 0.01$
- $\sigma_{\epsilon} = 0.10$
- $R = \left(1+0.01\right)^{1/12}$

**Question 2:** Calculate $H$ if the policy  $\ell_{t}=\left(\frac{(1-\eta)\kappa_{t}}{w}\right)^{\frac{1}{\eta}}$ from question 1 is followed. Choose $K$ so the approximation is good enough to not affect your results substantially.

In [21]:
# write your code here 

Next, we consider policies on the form:

$$

\ell_{t}=\begin{cases}
\ell_t^{\ast}  & \text{if }\left|\ell_{t-1}-\ell_t^{\ast} \right|>\Delta\\
\ell_{t-1} & \text{else }
\end{cases}
\\
\text{where}\,\,\ell_t^{\ast} = \left(\frac{(1-\eta)\kappa_{t}}{w}\right)^{\frac{1}{\eta}} \\

$$
With $\Delta \geq 0$ and $\Delta = 0$ being the previous policy.



**Question 3:** Calculate $H$ if the policy above was followed with $\Delta = 0.05$. Does it improve profitability?

In [22]:
# write your code here

**Question 4:** Find the optimal $\Delta$ maximizing $H$. Illustrate your result.

In [23]:
# write your code here


**Question 5:** Suggest an alternative policy you believe might improve profitability. Implement and test your policy.



In [24]:
# write your code here

## 3. <a id='toc3_'></a>[Problem 3: Global optimizer with refined multi-start](#toc0_)

We consider the Griewank function:

$$ f(\boldsymbol{x}) = \sum^n_{i=1} \frac{x^2_i}{4000}-\prod^n_{i=1}\cos\left(\frac{x_i}{\sqrt{i}}\right)+1$$

The **global minimum** of this function is $f(0,0) = 0$ (remember: $\cos(0)=1$).<br>
But the function also have a lot of **local minima**.

In [25]:
def griewank(x):
    return griewank_(x[0],x[1])
    
def griewank_(x1,x2):
    A = x1**2/4000 + x2**2/4000
    B = np.cos(x1/np.sqrt(1))*np.cos(x2/np.sqrt(2))
    return A-B+1

A **refined global optimizer with multi-start** is:

1. Choose *bounds* for $\mathbf{x}$ and *tolerance* $\tau > 0$.
2. Choose number of *warm-up iterations*, $\underline{K} > 0$ and *maximum number of iterations*, $K > \underline{K}$.
3. In each iteration for $k \in \{0,1,\dots,K-1\}$:

    A. Draw random $\mathbf{x}^k$ uniformly within chosen bounds.

    B. If $k < \underline{K}$ go to step E.

    C. Calculate $\chi^k = 0.50\cdot\frac{2}{1+\exp((k-\underline{K})/100)}$  

    D. Set $\mathbf{x}^{k0} = \chi^k \mathbf{x}^k + (1-\chi^k)\mathbf{x}^{\ast} $

    E. Run optimizer with $\mathbf{x}^{k0}$ as initial guess and $\mathbf{x}^{k\ast}$ as result.

    F. Set $\mathbf{x}^{\ast} = \mathbf{x}^{k\ast}$ if $k = 0$ or $f(\mathbf{x}^{k\ast}) < f(\mathbf{x}^{\ast})$

    G. If $f(\mathbf{x}^{\ast}) < \tau$ go to step 4.

4. Return the result $\mathbf{x}^{\ast}$.

As settings we choose:

* $x_1,x_2 \in  [-600,600]$
* $\tau = 10^{-8}$
* $\underline{K}=10$
* $K=1000$

The optimizer in Step 3.E is `BFGS` with a tolerance of $\tau$.

**Question 1:** Implement the refined global optimizer with multi-start. Illustrate how the effective initial guesses $\mathbf{x}^{k0}$ vary with the iteration counter $k$.

In [26]:
# write your code here

**Question 2:** Is it a better idea to set $\underline{K} = 100$? Is the convergence faster?

In [27]:
# write your code here